In [1]:
import importlib
%load_ext autoreload
%autoreload 2

In [2]:
import math
import pandas as pd
import numpy as np

def distance(x1,y1,x2,y2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

def getLinkCenter(x1,y1,x2,y2):
    return ((x1+x2)/2,(y1+y2)/2)

pd.set_option('display.max_columns', None)

#events=pd.read_csv('C:/Users/NRO_M4700_SSD_02/IdeaProjects/beam2_1/output/beamville/beamville__2019-09-02_21-54-09/ITERS/it.0/0.events.csv', low_memory=False)
#network=pd.read_csv('C:/Users/NRO_M4700_SSD_02/IdeaProjects/beam2_1/output/beamville/beamville__2019-09-02_21-54-09/network.csv.gz', low_memory=False)

events=pd.read_csv('https://beam-outputs.s3.amazonaws.com/output/sfbay/sfbay-smart-base-2010__2019-09-20_21-12-45/ITERS/it.15/15.events.csv.gz', low_memory=False)
network=pd.read_csv('https://beam-outputs.s3.amazonaws.com/pilates-outputs/15thSep2019/baseline/beam/sfbay-smart-base-pilates__2019-09-11_04-45-20/network.csv.gz', low_memory=False)

eventsAndNetwork = pd.merge(events, network,  how='left', left_on=['link'], right_on = ['linkId'])

linkCenterA=getLinkCenter(eventsAndNetwork['fromLocationX'],eventsAndNetwork['fromLocationY'],eventsAndNetwork['toLocationX'],eventsAndNetwork['toLocationY'])
eventsAndNetwork['center_x']=linkCenterA[0]
eventsAndNetwork['center_y']=linkCenterA[1]

In [3]:
eventsAndNetwork.tail()

,driver,vehicle,mode,vehicleType,links,numPassengers,length,primaryFuel,seatingCapacity,tollPaid,primaryFuelLevel,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,secondaryFuelLevel,time,type,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,cost,person,incentive,tollCost,netCost,departTime,price,score,fuel,duration,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,tourIndex,reason,link,facility,actType,linkId,linkLength,linkFreeSpeed,linkCapacity,numberOfLanes,linkModes,attributeOrigId,attributeOrigType,fromNodeId,toNodeId,fromLocationX,fromLocationY,toLocationX,toLocationY,center_x,center_y
10034471,NaN,body-6719440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214366.0,PersonLeavesVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6719440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10034472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214366.0,actstart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6719440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54722.0,NaN,Home,54722.0,269.770,12.517089,800.0,1.0,car;walk;bike,41586.0,unclassified,21790.0,27169.0,499672.709664,4.296886e+06,499421.934820,4.296785e+06,499547.322242,4.296835e+06
10034473,6624000,body-6624000,walk,BODY-TYPE-DEFAULT,NaN,0.0,120373.027114,0.0,0.0,0.0,2210000.0,38.714453,-122.900968,37.855147,-122.292187,0.0,214883.0,127656.0,NaN,0.0,None,Food,0.0,214883.0,PathTraversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10034474,NaN,body-6624000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214883.0,PersonLeavesVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6624000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10034475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214883.0,actstart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6624000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39546.0,NaN,Home,39546.0,1412.994,29.057528,2300.0,1.0,car;walk;bike,30497.0,primary,21233.0,21234.0,508330.247391,4.284217e+06,509333.691234,4.285213e+06,508831.969313,4.284715e+06


In [ ]:
class Person:
    def __init__(self):
        self.lastTimePersonEnteredVehicle=-1
        self.lastTimePersonLeftVehicle=-1
        self.lastTimePersonEnteredBodyVehicle=-1
        self.lastActEndTime=-1
        self.numberOfReplannings=0
        self.distanceTravelledPathTraversal=0
        self.durationTripBasedOnPathTraversal=0
        self.last_act_x=0
        self.last_act_y=0
        self.legId=0
        self.currentMode=""
        self.previousActType=""
        self.firstPathTraversalDepartureTime=-1
        self.lastPathTraversalArrivaltime=-1
        self.currentVehicle=""
        


def createPersonMilesTravelledTable(eventsAndNetwork):
    outputDf = pd.DataFrame(columns =['personId','legId','time','mode','numberOfReplannings','legDurationBasedOnActEndActStart','distanceTravelledPathTraversal','euclideanDistanceToPrevioustAct','previousActType'])

    withoutRidehail = eventsAndNetwork.loc[eventsAndNetwork["person"].str.contains('rideHail')==False,['person']]
    eventsFiltered = withoutRidehail[withoutRidehail["person"].str.contains('Driver')==False]
    
    #(time, personId, legId, mode, distance, duration, previousAct(home or work), eukleadean distance,internal_lastPathTraversal)
    personStates={}
    
    vehicles={}
    
    row_number = 0

    for index, row in eventsAndNetwork.iterrows():
        eventType=row['type']
        mode=row['mode']
        personId=row['person']
        time=row['time']
        vehicleId=row['vehicle']
        linkCenterX=row['center_x']
        linkCenterY=row['center_y']
        actType=row['actType']
        length=row['length']
        
        if row_number % 10000 == 0:
            print('Processing row' + str(row_number))
        row_number += 1

        
        if personId not in personStates:
            personStates[personId]=Person()
        
        person=personStates[personId]
        
        if eventType=='Replanning':
                person.numberOfReplannings=person.numberOfReplannings+1
            
        elif eventType=='ModeChoice':
            person.currentMode=mode
        elif eventType=='actstart':
            outputDf = outputDf.append({'personId':personId, 
                'legId':person.legId, 
                'time':time, 
                'mode':person.currentMode, 
                'numberOfReplannings':person.numberOfReplannings,
                'legDurationBasedOnActEndActStart':time-person.lastActEndTime,
                #'durationBasedOnFirstAndLastLegPathTraversalTime':lastPathTraversalArrivaltime-firstPathTraversalDepartureTime,
               #'durationTripBasedOnPathTraversal':durationTripBasedOnPathTraversal,
               'distanceTravelledPathTraversal':person.distanceTravelledPathTraversal,
                'euclideanDistanceToPrevioustAct':distance(person.last_act_x,person.last_act_y,linkCenterX,linkCenterY),
               'previousActType':person.previousActType}, ignore_index=True)


            legId=person.legId+1
            personStates[personId]=Person()
            personStates[personId].legId=legId

        elif eventType=='actend':
            person.lastActEndTime=time
            person.last_act_x=linkCenterX
            person.last_act_y=linkCenterY
            person.previousActType=actType
        elif eventType=='PersonEntersVehicle': 
            
            if ('rideHail' in personId) | ('Driver' in personId):
                continue
            
            person.lastTimePersonEnteredVehicle=time
            person.currentVehicle=vehicleId
            
            if vehicleId not in vehicles:
                vehicles[vehicleId]=set()
            
            vehicles[vehicleId].add(personId)
        elif eventType=='PersonLeavesVehicle':
            lastTimePersonLeftVehicle=time
            person.currentVehicle=""
            
            if vehicleId not in vehicles:
                vehicles[vehicleId]=set()
            
            #if (personId not in vehicles[vehicleId]):
                #print("trying to unboard person, which seems to have not boarded vehicle:",personId)
            
            #print(vehicles[vehicleId])
            vehicles[vehicleId].discard(personId)
        elif eventType=='PathTraversal':
            
            if 'body' in vehicleId:
                personId=vehicleId.split('-')[1]
                personStates[personId].distanceTravelledPathTraversal=personStates[personId].distanceTravelledPathTraversal+length
            else:
                if vehicleId not in vehicles:
                    vehicles[vehicleId]=set()

                for personId in vehicles[vehicleId]:
                    #if personId=='5':
                    #    print(vehicleId,length)
                    personStates[personId].distanceTravelledPathTraversal=personStates[personId].distanceTravelledPathTraversal+length

    return outputDf

outputDf=createPersonMilesTravelledTable(eventsAndNetwork)
outputDf.to_csv("personMilesTravelled.csv")  

Processing row0
Processing row10000
Processing row20000
Processing row30000
Processing row40000
Processing row50000
Processing row60000
Processing row70000
Processing row80000
Processing row90000
Processing row100000
Processing row110000
Processing row120000
Processing row130000
Processing row140000
Processing row150000
Processing row160000
Processing row170000
Processing row180000
Processing row190000
Processing row200000
Processing row210000
Processing row220000
Processing row230000
Processing row240000
Processing row250000
Processing row260000
Processing row270000
Processing row280000
Processing row290000
Processing row300000
Processing row310000
Processing row320000
Processing row330000
Processing row340000
Processing row350000
Processing row360000
Processing row370000
Processing row380000
Processing row390000
Processing row400000
Processing row410000
Processing row420000
Processing row430000
Processing row440000
Processing row450000
Processing row460000
Processing row470000
Proces